# Finale Aufgabe für Praktikum Deep Learning <br>Textgenerierung mit RNN: Modelltraining

* **Name:** Fabian Schotte
* **Email:** fabian.schotte@rwu.de
* **Matrikelnummer:** 35604
* **Studiengang:** Angewandte Informatik

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pylab as plt
from work import models
import time 

os.makedirs("models", exist_ok=True)

## Vorbereitung


### Laden der Trainingsdaten
Hier werden die Trainings- und Testdaten der Kaggle Sentiment Analyis aus deren CSV-Dateien ausgelesen und die Inhalte der Spalte `text` zu dem String `kaggle_text` zusammengefasst.

In [ ]:
df_train = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_train.csv', encoding='utf-8', encoding_errors='replace')
df_test = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_test.csv', encoding='utf-8', encoding_errors='replace')

kaggle_text_train = df_train['text'].str.cat(sep='\n')
kaggle_text_test = df_test['text'].str.cat(sep='\n')
# kaggle_text = kaggle_text_train + '\n' + kaggle_text_test
kaggle_text = kaggle_text_train
# kaggle_text = kaggle_text_test

print(kaggle_text[:500])

Im nächsten Codeblock wird ein Set der einzigartigen Charaktere im String `kaggle_text` mit dem Namen `vocab` erstellt. Ebenso werden die darin vorhandenen Charaktere ausgegeben und die Länge des Sets ausgegeben.

In [ ]:
vocab = sorted(set(kaggle_text))
print(vocab)
print(f"vocab size = {len(vocab)}")

## Preprocessing
Im folgenden Codeblock wird ein Beispieltext zu einer Liste von Charakteren aufgeteilt und ausgegeben. Diese List wird als `chars` gespeichert.

In [ ]:
example_texts = ['hello world', 'hello world']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

In der Funktion `ids_from_chars` werden die gegebenen Charaktere in eine zugeordnete Zahl für das Training und die Vorhersage umgewandelt.
Die Funktion `chars_from_ids` funktioniert genau umgekehrt, indem hier die gegebenen Zahlen in die dazugehörigen Charaktere umgewandelt werden.
Hier werden auch die beiden Methoden für den Beispieltext aus `chars` durchgeführt und ausgegeben. Zur Vollständigkeit werden die Charaktere, die aus den Zahlenwerten generiert wurden, wieder zu einem String zusammengefügt.

In [ ]:
ids_from_chars = keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
print(ids)

chars_from_ids = keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
print(chars)

tf.strings.reduce_join(chars, axis=-1).numpy()

In `text_from_ids(ids)` wird genau die Operation für das Zusammenfügen der Charaktere zu einem String ausgeführt.

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Hier wird die zuvor genannte Funktion zur Berechnung der Zahlenwerte aus den Charakteren `kaggle_text` verwendet, um die Trainingsdaten als Zahlenwerte zu erhalten. Diese werden in `all_ids` gespeichert.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(kaggle_text, 'UTF-8'))
all_ids

Als nächstes wird ein Dataset aus den Zahlenwerten generiert. Die ersten 10 Werte des Datasets werden als Charaktere ausgegeben.

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Mit `seq_lenth` wird die Länge der Sequenz definiert, mit dem die Modelle trainiert werden.

In [ ]:
seq_length = 100

Im nächsten Schritt werden die Sequenzen mit Hilfe von `seq_lenth` generiert. Die Daten für die Sequenzeb kommen aus dem zuvor angelegten Dataset `ids_dataset`.
Dazu werden auch die Charaktere der ersten Sequenz zuerst einzeln und dann als Text ausgegeben.

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq).numpy())
for seq in sequences.take(1):
  print(text_from_ids(seq).numpy())

In `split_input_target(sequence)` werden die Inputs und Target Labels der Sequenz generiert

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

Im folgenden Code wird die zuvor erstellte Methode `split_input_target` auf die Sequenzen der Kaggle Trainingsdaten angewendet und die Länge des Datasets ausgegeben

In [ ]:
kaggle_dataset = sequences.map(split_input_target)
len(kaggle_dataset)

Als nächstes werden Beispiele für die Input und Labels als Text und mit dessen Shape ausgegeben.

In [ ]:
for input_example, target_example in kaggle_dataset.take(1):
    print("Input:", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
    print("Input shape:", input_example.shape)
    print("Target shape:", target_example.shape)

Als nächstes werden folgende Variablen für das Training definiert:
- Mit `BATCH_SIZE` wird definiert, wie viele Input-Label-Paare in einer Epoche verarbeitet werden. Hier sind dies 150
- Mit `BUFFER_SIZE` wird die Anzahl der Elemente des Datasets definiert, die zufällig gemischt werden, bevor sie in die Batches eingeteilt werden.

Dieses Variablen werden hier auch auf das Dataset `kaggle_dataset` angewendet und dieses Dataset wird ebenfalls ausgegeben. 

In [ ]:
BATCH_SIZE = 150
BUFFER_SIZE = 1000

kaggle_dataset = (
    kaggle_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)
kaggle_dataset

### 1. GRU-Modell

Hier werden die Variablen für das erste GRU-Modell definiert.
`vocab_size_1` wird auf die Anzahl der einzigartigen Zeichen gesetzt und der Vektor für die Embedding-Schicht wird in `embedding_dim_1` auf 256 gesetzt. Die Anzahl der Neuronen wird in `rnn_units` auf 2048 gesetzt.

In [ ]:
vocab_size_1 = len(ids_from_chars.get_vocabulary())
embedding_dim_1 = 256
rnn_units_1 = 2048

Hier wird das erste GRU-Modell definiert. Als erstes wird ein Input Layer durchlaufen, worauf ein Embedding Layer folgt. Im Anschluss wird ein Stacked GRU-Layer verwendet, dass heißt es werden zwei GRU-Layers hintereinander verwendet. Dieses kann durch diesen Aufbau bessere Modelle trainineren, z.B. ein besseres Verständnis von Grammatikregeln.
Danach wird ein Dense Layer für den Output verwendet.

In [ ]:
inputs_1 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_1 = keras.layers.Embedding(input_dim=vocab_size_1, output_dim=embedding_dim_1)(inputs_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(embedding_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(gru_1)
outputs_1 = keras.layers.Dense(units=vocab_size_1, activation='softmax')(gru_1)

gru_model_1 =  keras.Model(inputs=inputs_1, outputs=outputs_1)

#### Testen des Modells

Hier werden zum Test die Dimension des ersten Batches as dem Dataset ausgegeben. Die Ausgabe gibt die Batch Size, Sequence Length und die Vocab Size des ersten GRU-Modells an.

In [ ]:
for input_example_batch_1, target_example_batch_1 in kaggle_dataset.take(1):
    example_batch_predictions_model_1 = gru_model_1(input_example_batch_1)
    print(example_batch_predictions_model_1.shape, "# (batch_size, sequence_length, vocab_size_1)")

Hier wird die Zusammenfassung der wichtigsten Informationen des ersten GRU-Modells angezeigt.

In [ ]:
gru_model_1.summary()

Im nächsten Codeblock wird die Zufallsauswahl der Zahlenwerte durch das erste GRU-Modell getestet. Die finale Ausgabe ist das Array mit den Zufallswerten, in dem unnötige Dimensionen entfernt wurden.

In [ ]:
sampled_indices_gru_model_1 = tf.random.categorical(example_batch_predictions_model_1[0], num_samples=1)
sampled_indices_gru_model_1 = tf.squeeze(sampled_indices_gru_model_1, axis=-1).numpy()
sampled_indices_gru_model_1

In dem folgenden Code werden die Zahlenwerte des ersten Batches genommen und als Text ausgegeben.
Es wird ebenfalls der Text ausgegeben, der durch die Predictions aus dem GRU-Modell 1 erzeugt wurden.

In [ ]:
print("Input:\n", text_from_ids(input_example_batch_1[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_1).numpy())

### Training

#### Loss

Hier wird die Verlustfunktion definiert. In diesem Fall handelt es sich um Sparse Categorial Crossentropy mit dem Parameter `from_logits` als `True`.

In [ ]:
loss_1 = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

Hier wird jetzt der Loss auf das GRU-Modell angewendet, indem die Variablen `target_example_batch_1` und `example_batch_predictions_1` als Parameter verwendet werden. Die Ausgabe ist hierzu einmal die Dimensionen des Prediction Modells 1 sowie die Ausgabe des Mean Loss des ersten Modells.

In [ ]:
example_batch_mean_loss_model_1 = loss_1(target_example_batch_1, example_batch_predictions_model_1)
print("Prediction shape: ", example_batch_predictions_model_1.shape, " # (batch_size, sequence_length, vocab_size_1)")
print("Mean loss:        ", example_batch_mean_loss_model_1)

Als nächstes wird der Exponentialwert des Mean Loss berechnet. Dieser Sollte in etwa ähnlich zur Vocab Size sein.

In [ ]:
tf.exp(example_batch_mean_loss_model_1).numpy()

#### Optimizer

Als nächstes wird das Model kompiliert. Dabei wird der Optimizer `adam`, als Metrik die Accuracy verwendet. Damit es im Ablauf des Trainings keine Fehler gibt, wird der Parameter `run_eagerly` mit dem Wert `True` verwendet.

In [ ]:
gru_model_1.compile(optimizer='adam', loss=loss_1, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

Hier wird die Early Stopping Callback Funktion für das erste GRU-Modell definiert, damit das Training gestoppt wird, sobald das Modell einen optimalen Wert erreicht hat. Hier wird das Training gestoppt, sobald das Modell sich nicht innerhalb von 2 Epochen um 0,002 verbessert hat.

In [ ]:
early_stopping_gru_model_1 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

Als nächstes wird die Checkpoints Callback definiert, damit nach jeder Epoche die Gew ichtung des Modells gespeichert wird.

In [ ]:
checkpoint_dir = './work/training_checkpoints/gru_model_1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_1=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

Für das Training wird eine EPOCHS Konstante definiert, hier ist dies 30 Epochen, dass heißt, dass das Modell 30 mal komplett durchlaufen wird.

In [ ]:
EPOCHS_1 = 30

Hier wird das erste Modell trainiert. In dem Modell werden das vorher definierte Dataset, die Epochenzahl sowie die Callback Funktionen verwendet. Weiterhin wird für einen späteren Vergleich der Modelle eine Zeitstoppung der gesamten Trainingszeit durchgeführt.

In [ ]:
start = time.perf_counter()
gru_model_1_history = gru_model_1.fit(kaggle_dataset, epochs=EPOCHS_1, callbacks=[checkpoint_callback_gru_model_1, early_stopping_gru_model_1])
end = time.perf_counter()
gru_model_1_training_time = end - start

#### Speichern

Im letzten Schritt für das erste GRU-Modell wird dieses gespeichert, damit es im Notebook für die Textgenerierung verwendet werden kann.

In [ ]:
gru_model_1.save('work/models/gru_model_1.keras')

### 2. GRU-Modell

Hier werden wie beim ersten GRU-Modell für das zweite GRU-Modell auch die Vocab Size, die Embedding Dimension sowie die Anzahl der Neuronen festgelegt, diese Anzahl beträgt hier 1024 anstatt der 2048 aus dem ersten Modell.

In [ ]:
vocab_size_2 = len(ids_from_chars.get_vocabulary())
embedding_dim_2 = 256
rnn_units_2 = 1024

Hier wird das zweite GRU-Modell definiert, im Vergleich zum ersten Modell hat dieses zwischen den beiden GRU-Layers eine weitere Schicht. Dies ist das Dropout Layer, dass bewirkt, dass das Modell Overfitting vermeidet, indem 20% der Neuronen auf 0 gesetzt werden. Mit diesem Layer soll das zweite GRU-Layer bessere Ergebnisse erzeugen.

In [ ]:
inputs_2 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_2 = keras.layers.Embedding(input_dim=vocab_size_2, output_dim=embedding_dim_2)(inputs_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(embedding_2)
dropout_2 = keras.layers.Dropout(0.2)(gru_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(dropout_2)
outputs_2 = keras.layers.Dense(units=vocab_size_2, activation='softmax')(gru_2)

gru_model_2 = keras.Model(inputs=inputs_2, outputs=outputs_2)

#### Testen des Modells

Hier wird wie auch beim ersten GRU-Modell das erste Batch des Datasets abgerufen und dessen Dimensionen angezeigt.

In [ ]:
for input_example_batch_2, target_example_batch_2 in kaggle_dataset.take(1):
    example_batch_predictions_gru_model_2 = gru_model_2(input_example_batch_2)
    print(example_batch_predictions_gru_model_2.shape, "# (batch_size, sequence_length, vocab_size_2)")

Der Vorgang ist hier ebenfalls der gleiche wie beim ersten Modell, indem die Zusammenfassung des zweiten GRU-Modells ausgegeben wird.

In [ ]:
gru_model_2.summary()

Hier wird wie im ersten GRU-Modell auch die Prediction des Modells getestet.

In [ ]:
sampled_indices_gru_model_2 = tf.random.categorical(example_batch_predictions_gru_model_2[0], num_samples=1)
sampled_indices_gru_model_2 = tf.squeeze(sampled_indices_gru_model_2, axis=-1).numpy()
sampled_indices_gru_model_2

Hier wird wie auch beim ersten GRU-Modell einmal der Text des ersten Batches ausgegeben und die Vorhersage der nächsten Zeichen des zweiten GRU-Modells.

In [ ]:
print("Input:\n", text_from_ids(input_example_batch_2[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_2).numpy())

### Training

#### Loss

Hier wird die auch beim ersten GRU-Modell die Loss Funktion definiert mit der gleichen Konfiguration wie im ersten Modell.

In [ ]:
loss_2 = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_model_2 = loss_2(target_example_batch_2, example_batch_predictions_gru_model_2)
print("Prediction shape: ", example_batch_predictions_gru_model_2.shape, " # (batch_size, sequence_length, vocab_size_2)")
print("Mean loss:        ", example_batch_mean_loss_model_2)

In [ ]:
tf.exp(example_batch_mean_loss_model_1).numpy()

#### Optimizer

Hier wird das zweite GRU-Modell kompiliert und dazu werden die gleichen Parameter wie beim ersten GRU-Modell verwendet und die Loss Funktion `loss_2`.

In [ ]:
gru_model_2.compile(optimizer='adam', loss=loss_2, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

Als nächstes wird die Early Stopping Callback eingerichtet, diese hat die gleiche Konfiguration wie aus dem ersten GRU-Modell.

In [ ]:
early_stopping_model_2 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

Die Checkpoint Callback hat hier die gleiche Konfiguration wie die für das erste GRU-Modell.

In [ ]:
checkpoint_dir = './work/training_checkpoints/gru_model_2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

Das Training des zweiten GRU-Modells wird auch mit 30 Epochen durchgeführt.

In [ ]:
EPOCHS_2 = 30

Als nächstes wird das Training des zweiten GRU-Modells durchgeführt und die gesamte Trainingszeit gemessen.

In [ ]:
start = time.perf_counter()
gru_model_2_history = gru_model_2.fit(kaggle_dataset, epochs=EPOCHS_2, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
gru_model_2_training_time = end - start

### Speichern

Das zweite GRU-Modell wird hier gespeichert.

In [ ]:
gru_model_2.save('work/models/gru_model_2.keras')

## Bewertung und Vergleich der GRU-Modelle

Als nächstes soll ein LSTM-Modell auf der Struktur des besseren GRU-Modell erstellt werden. Damit diese Entscheidung getroffen werden kann, werden die beiden Modelle hier auf Loss, Accuracy, Perplexity und Trainingszeit vergleichen.

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()

print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")


### LSTM-Modell

Da im ersten Durchlauf des Notebooks das erste GRU-Modell die besseren Ergebnisse erzielt hatte, wird dessen Aufbau für die Erstellung des LSTM-Modells verwendet. Dazu wird zum einen die Neuronenanzahl von 2048 verwendet.

In [ ]:
vocab_size_lstm = len(ids_from_chars.get_vocabulary())
embedding_dim_lstm = 256
rnn_units_lstm = 2048

Hier wird das LSTM-Modell für den zweiten Teil der Aufgabe erstellt. Da das erste GRU-Modell besser war, wird hier wie in dem ersten GRU-Modell ein Stacked-LSTM-Modell erstellt.

In [ ]:

inputs_lstm = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_lstm = keras.layers.Embedding(input_dim=vocab_size_lstm, output_dim=embedding_dim_lstm)(inputs_lstm)
lstm, hidden_state_1, cell_state_1 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(embedding_lstm)
lstm, hidden_state_2, cell_state_2 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(lstm)
outputs_lstm = keras.layers.Dense(units=vocab_size_lstm, activation='softmax')(lstm)

lstm_model = keras.Model(inputs=inputs_lstm, outputs=outputs_lstm)

#### Testen des Modells

Wie auch in den beiden GRU-Modellen, wird hier für das LSTM-Modell die Dimension der Modell Prediction ausgegeben.

In [ ]:
for input_example_batch_lstm, target_example_batch_lstm in kaggle_dataset.take(1):
    example_batch_predictions_lstm_model = lstm_model(input_example_batch_lstm)
    print(example_batch_predictions_lstm_model.shape, "# (batch_size, sequence_length, vocab_size)")

Hier wird die Zusammenfassung des LSTM-Modells ausgegeben.

In [ ]:
lstm_model.summary()

Wie bei den beiden ersten Modellen wird hier für das LSTM-Modell die Prediction des Modells zum Test ausgegeben.

In [ ]:
sampled_indices_lstm_model = tf.random.categorical(example_batch_predictions_lstm_model[0], num_samples=1)
sampled_indices_lstm_model = tf.squeeze(sampled_indices_lstm_model, axis=-1).numpy()
sampled_indices_lstm_model

Hier gibt es den gleichen Vorgang wie bei den beiden GRU-Modellen: Zuerst wird das erste Input Sample Batch des Datasets ausgegeben, dann wird die Vorhersage der Zeichen des LSTM-Modells ausgegeben.

In [ ]:
print("Input:\n", text_from_ids(input_example_batch_lstm[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_lstm_model).numpy())

### Training

#### Loss

Als nächstes wird wie auch bei den GRU-Modellen die Loss Funktion definiert.

In [ ]:
loss_lstm = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

Hier wird jetzt der Loss auf das LSTM-Modell angewendet, indem die Variablen `target_example_batch_lstm` und `example_batch_predictions_lstm_model` als Parameter verwendet werden. Die Ausgabe ist hierzu einmal die Dimensionen der Predictions des LSTM-Modell sowie die Ausgabe des Mean Loss des LSTM-Modells.

In [ ]:
example_batch_mean_loss_lstm_model = loss_lstm(target_example_batch_lstm, example_batch_predictions_lstm_model)
print("Prediction shape: ", example_batch_predictions_lstm_model.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss_lstm_model)

Als nächstes wird der Exponentialwert des Mean Loss aus dem LSTM-Modell berechnet. Dieser Sollte in etwa ähnlich zur Vocab Size sein.

In [ ]:
tf.exp(example_batch_mean_loss_lstm_model).numpy()

#### Optimizer

Hier wird das LSTM-Modell kompiliert und dazu werden die gleichen Parameter wie beim ersten GRU-Modell verwendet und die Loss Funktion `loss_lstm`.

In [ ]:
lstm_model.compile(optimizer='adam', loss=loss_lstm, metrics=['accuracy'], run_eagerly=True)

#### Konfiguration von Checkpoints

Zum Speichern der Gewichtungen des LSTM-Modells wird hier wie auch bei den GRU-Modellen eine Checkpoint Callback eingerichtet.

In [ ]:
checkpoint_dir = './work/training_checkpoints/lstm_model'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

Die Epochenzahl wird für das Training des LSTM-Modells ebenfalls auf 30 gesetzt.

In [ ]:
EPOCHS_LSTM = 30

Das LSTM-Modell wird hier trainiert und dazu wird ebenfalls die Trainingszeit gemessen.

In [ ]:
start = time.perf_counter()
lstm_model_history = lstm_model.fit(kaggle_dataset, epochs=EPOCHS_LSTM, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
lstm_model_training_time = end - start

#### Speichern

Das LSTM-Modell wird hier gespeichert.

In [ ]:
lstm_model.save('work/models/lstm_model.keras')

## Vergleich aller Modelle

Zum Schluss wird der Vergleich aller drei Modelle angezeigt mit den gleichen Parametern wie auch bei dem Vergleich der beiden GRU-Modellen.

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']
loss3 = lstm_model_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']
accuracy3 = lstm_model_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)
perplexity3 = np.exp(loss3)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].plot(loss3, label="LSTM Model", linestyle='-', color='green')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].plot(accuracy3, label="LSTM Model", linestyle='-', color='green')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].plot(perplexity3, label="LSTM Model", linestyle='-', color='green')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()
print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print(f" LSTM Model:  {loss3[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print(f" LSTM Model:  {accuracy3[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print(f" LSTM Model:  {perplexity3[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")
print(f" LSTM Model:  {lstm_model_training_time:2f}s")